<a href="https://colab.research.google.com/github/Krizzna69/cod-soft/blob/main/sms_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np        # For numerical operations
import pandas as pd       # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns       # For data visualization

import warnings
warnings.filterwarnings('ignore')

# handling imbalanced datasets and performing resampling techniques
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv('/content/sms-spam-collection-dataset/spam.csv', encoding='latin1')
data.head()

data.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

data.info()

data.rename(columns = {'v1': 'target', 'v2': 'Message'}, inplace = True)

from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()
data['target'] = en.fit_transform(data['target'])

data.head()

data = data.drop_duplicates(keep = 'first')
l = data['target'].value_counts()
colors = ['#8BC34A','#B2EBF2']

fig = plt.subplots(nrows = 1,ncols = 2,figsize = (20,5))
plt.subplot(1,2,1)
l.plot.pie(explode=[0,0.1],autopct='%1.1f%%',shadow=True, labels=['Ham', 'Spam'], colors=colors)
plt.title('Target (%)')


plt.subplot(1,2,2)
ax = sns.countplot(x='target',data = data, palette = colors,edgecolor = 'black', width=0.4)
for rect in ax.patches:
    ax.text(rect.get_x() + rect.get_width() / 2, rect.get_height(), rect.get_height(), horizontalalignment='center', fontsize = 11)
ax.set_xticklabels(['Ham', 'Spam'])
plt.title('Number of Target')
plt.show()

x = data['Message']
y = data['target']

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)

x_extraction = feature_extraction.fit_transform(x)

print(x_extraction)

# Performing oversampling and undersampling using SMOTE and RandomUnderSampler techniques to address class imbalance

over = SMOTE(sampling_strategy = 1)
under = RandomUnderSampler(sampling_strategy = 0.4)
f1 = x_extraction
t1 = y

steps = [('under', under),('over', over)]
pipeline = Pipeline(steps=steps)
f1, t1 = pipeline.fit_resample(f1, t1)
Counter(t1)

# data splitting, model evaluation metrics, cross-validation, hyperparameter tuning, and classification performance
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve


# necessary classifiers for building machine learning models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

x_train, x_test, y_train, y_test = train_test_split(f1, t1, test_size=0.3, random_state=101)


# Defining a function to train and evaluate a machine learning classifier model

def model(classifier,x_train,y_train,x_test,y_test):

    classifier.fit(x_train,y_train)
    prediction = classifier.predict(x_test)
    cv = RepeatedStratifiedKFold(n_splits = 10,n_repeats = 3,random_state = 1)
    print("Cross Validation Score : ",'{0:.2%}'.format(cross_val_score(classifier,x_train,y_train,cv = cv,scoring = 'roc_auc').mean()))
    print("ROC_AUC Score : ",'{0:.2%}'.format(roc_auc_score(y_test,prediction)))



# Defining a function to evaluate the performance of a machine learning classifier model
def model_evaluation(classifier,x_test,y_test):

    # Confusion Matrix
    cm = confusion_matrix(y_test,classifier.predict(x_test))
    names = ['True Neg','False Pos','False Neg','True Pos']
    counts = [value for value in cm.flatten()]
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cm,annot = labels,cmap = colors,fmt ='')

    # Classification Report
    print(classification_report(y_test,classifier.predict(x_test)))





# Defining a function to plot the Receiver Operating Characteristic (ROC) curve
def plot_roc_curve(y_true, y_scores):
    # Calculate the false positive rate (FPR) and true positive rate (TPR)
    fpr, tpr, _ = roc_curve(y_true, y_scores)

    # Calculate the area under the ROC curve (AUC)
    auc = roc_auc_score(y_true, y_scores)

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (AUC = {:.2f})'.format(auc))
    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line (random classifier)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()


    # Initializing an XGBoost classifier with specific hyperparameters

classifier_xgb = XGBClassifier(learning_rate= 0.01,max_depth = 3,n_estimators = 1000)


# Training, evaluating, and evaluating the performance of an XGBoost classifier model

model(classifier_xgb,x_train,y_train,x_test,y_test)
model_evaluation(classifier_xgb,x_test,y_test)

# Making predictions using the XGBoost classifier and plotting the ROC curve

prediction_xgb = classifier_xgb.predict(x_test)
plot_roc_curve(y_test, prediction_xgb)# Initializing a Logistic Regression classifier with specific hyperparameters

classifier_lr = LogisticRegression(random_state = 0,C=10,penalty= 'l2')# Training, evaluating, and evaluating the performance of a Logistic Regression classifier model

model(classifier_lr,x_train,y_train,x_test,y_test)
model_evaluation(classifier_lr,x_test,y_test)# Making predictions using the Logistic Regression classifier and plotting the ROC curve

prediction_lr = classifier_lr.predict(x_test)
plot_roc_curve(y_test, prediction_lr)

classifier_svc = SVC(kernel = 'linear',C = 0.1)

model(classifier_svc,x_train,y_train,x_test,y_test)
model_evaluation(classifier_svc,x_test,y_test)

prediction_svc = classifier_svc.predict(x_test)
plot_roc_curve(y_test, prediction_svc)

classifier_rf = RandomForestClassifier(max_depth = 4,random_state = 0)
prediction_rf = classifier_rf.predict(x_test)
plot_roc_curve(y_test, prediction_rf)

classifier_knn = KNeighborsClassifier(leaf_size = 1, n_neighbors = 3,p = 1)

model(classifier_knn,x_train,y_train,x_test,y_test)
model_evaluation(classifier_knn,x_test,y_test)


prediction_knn = classifier_knn.predict(x_test)
plot_roc_curve(y_test, prediction_knn)